In [1]:
import sys, os
import netCDF4 as nc
sys.path.append('../src/')
from Biologging_Toolkit.wrapper import Wrapper
from Biologging_Toolkit.utils import *

/home6/grosmaan/Documents/SES_tags/notebooks/../src/Biologging_Toolkit


In [5]:
depid = 'ml17_280a'
path = '/home6/grosmaan/Documents/data'
sens_path = os.path.join(path, 'CTD', depid, depid+'sens5.nc')
raw_path = os.path.join(path, 'individus', depid, 'raw')

In [7]:
inst = Wrapper(depid, 
            path = path 
              )

inst.dt = 5   # Timestep in seconds of final data structure

In [8]:
inst.ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: Processed dataset for ml17_280a
    subtitle: NetCDF structure storing processed data from the ml17_280a individual using a 3 s timestep
    sampling_rate: 3
    sampling_rate_units: Seconds
    dimensions(sizes): time(528420)
    variables(dimensions): float64 time(time), float32 depth(time), float64 dives(time), float32 lat(time), float32 lon(time), float64 azimuth(time), float64 elevation_angle(time), float64 bank_angle(time), float64 vertical_azimuth(time), float64 inertial_speed(time), float64 jerk(time)
    groups: 

### Create reference time data

Enter either a sens5 path containing existing time data or a time vector in POSIX timestamps (UTC)
In both cases, the data will be resampled to match inst.dt

Change overwrite to True if time data already exists and you wish to overwrite it.

In [8]:
inst.create_time(time_path = sens_path, overwrite = False)

In [9]:
inst.create_time(time_data = [151878955, 154324489], overwrite = False)

In [10]:
inst.ds['time'][:]

masked_array(data=[1.50733444e+09, 1.50733444e+09, 1.50733444e+09, ...,
                   1.50997879e+09, 1.50997879e+09, 1.50997879e+09],
             mask=False,
       fill_value=1e+20)

### Load GPS data

Enter time, latitude and longitude data (all the same size) and it will be resampled to match reference time data

Change overwrite to True if GPS data already exists and you wish to overwrite it.

In [10]:
trk_path = os.path.join('/run/media/grosmaan/LaCie/individus_brut/CTD', depid, depid+'trk.nc')
trk = nc.Dataset(trk_path)

In [37]:
time, lat, lon = trk['POS'][:].data
time =  time + get_start_time_sens(trk.dephist_deploy_datetime_start)

In [39]:
inst.create_gps(time_data=time, lat_data=lat, lon_data=lon, overwrite = True)

In [42]:
inst.ds['lat'][:], inst.ds['lon'][:]

(masked_array(data=[-47.307076, -47.30733 , -47.3073  , ..., -43.50783 ,
                    -43.507843, -43.50785 ],
              mask=False,
        fill_value=1e+20,
             dtype=float32),
 masked_array(data=[67.76784 , 67.76795 , 67.767944, ..., 53.032776,
                    53.032837, 53.032894],
              mask=False,
        fill_value=1e+20,
             dtype=float32))

### Add any other data

You can now add any other data to structure, here we add depth as an example

In [10]:
var_name = 'depth'
sens = nc.Dataset(sens_path)
var_data = sens['P'][:].data

length = len(sens['P'][:])
ds_sr = sens['P'].sampling_rate
time_data = get_start_time_sens(sens.dephist_device_datetime_start) + np.arange(0, length/ds_sr, np.round(1/ds_sr,2))

In [ ]:
inst.create_variable(var_name, var_data, time_data, overwrite = False)

In [43]:
inst.ds.close()